In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis' )

import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.ndimage.interpolation import shift
from scipy.signal import decimate
from matplotlib import pyplot as plt
from scipy import stats
from scipy.stats import sem
import pandas as pd
import pickle
from matplotlib import colors, cm
from utils.post_processing_utils import remove_exps_after_manipulations, remove_bad_recordings
from matplotlib.lines import Line2D
from utils.regression.linear_regression_utils import *

In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
experiment_record = pd.read_csv('W:\\photometry_2AC\\experimental_record.csv')
experiment_record['date'] = experiment_record['date'].astype(str)

In [3]:
param_names = ['high cues', 'low cues', 'ipsi choices', 'contra choices', 'rewards', 'no rewards']

In [21]:
mouse_ids = [ 'SNL_photo16', 'SNL_photo17', 'SNL_photo18', 'SNL_photo21', 'SNL_photo22', 'SNL_photo26']
site = 'tail'

In [11]:
mouse_ids = [ 'SNL_photo28', 'SNL_photo29', 'SNL_photo30', 'SNL_photo31', 'SNL_photo32', 'SNL_photo33', 'SNL_photo34', 'SNL_photo35']
site = 'Nacc'

In [16]:
mouse_ids = [  'SNL_photo21']
site = 'tail'

In [15]:
#non_manipulation_experiments = remove_exps_after_manipulations(experiment_record, mouse_ids)
#clean_experiments = remove_bad_recordings(non_manipulation_experiments)
#all_experiments_to_process = clean_experiments[(clean_experiments['mouse_id'].isin(mouse_ids) ) & (clean_experiments['recording_site'] == site)]
#experiments_to_process = get_first_x_sessions(all_experiments_to_process).reset_index(drop=True)

In [24]:
clean_experiments = remove_exps_after_manipulations(experiment_record, mouse_ids)
all_experiments_to_process = clean_experiments[(clean_experiments['mouse_id'].isin(mouse_ids)) & (clean_experiments['recording_site'] == site)]
experiments_to_process = get_first_x_sessions(all_experiments_to_process).reset_index(drop=True)

removing SNL_photo16: ['20200313' '20200315' '20200318' '20200320' '20200322']
removing SNL_photo17: ['20200311' '20200313' '20200317']
removing SNL_photo18: ['20200318' '20200320' '20200322']
removing SNL_photo22: ['20200829' '20200830' '20200831' '20200908' '20200910' '20200911'
 '20200915' '20200917' '20200918' '20200921' '20201007' '20201008'
 '20201009']
removing SNL_photo26: ['20200829' '20200830' '20200831' '20200908' '20200910' '20200911'
 '20200915' '20200917' '20200918' '20200921' '20201007' '20201008'
 '20201009']
removing SNL_photo21: ['20200829' '20200830' '20200831' '20200908' '20200910' '20200911'
 '20200915' '20200917' '20200918' '20200921' '20201007' '20201008'
 '20201009']


In [25]:
experiments_to_process

,mouse_id,date,recording_site,fiber_side,experiment_notes,recording_notes,include,Unnamed: 7
0,SNL_photo16,20200210,tail,left,NaN,NaN,NaN,NaN
1,SNL_photo16,20200213,tail,left,NaN,NaN,NaN,NaN
2,SNL_photo16,20200218,tail,left,NaN,NaN,NaN,NaN
3,SNL_photo17,20200204,tail,right,NaN,NaN,NaN,NaN
4,SNL_photo17,20200206,tail,right,NaN,NaN,NaN,NaN
5,SNL_photo17,20200208,tail,right,NaN,NaN,NaN,NaN
6,SNL_photo18,20200223,tail,left,NaN,NaN,NaN,NaN
7,SNL_photo18,20200226,tail,left,NaN,NaN,NaN,NaN
8,SNL_photo18,20200228,tail,left,NaN,NaN,NaN,NaN
9,SNL_photo21,20200805,tail,left,NaN,lots of motion artefacts,no,NaN


In [19]:
exp_numbers = []
mice = []
for index, experiment in experiments_to_process.iterrows():
    mouse = experiment['mouse_id']
    date = experiment['date']
    saving_folder = 'W:\\photometry_2AC\\processed_data\\' + mouse + '\\'
    save_filename = mouse + '_' + date + '_' + 'linear_regression_kernels_no_repeated_cues_both_cues.p'
    fiber_side = experiment['fiber_side']
    num_recordings = experiments_to_process[experiments_to_process['mouse_id'] == mouse].shape
    sorted_exps = pd.to_datetime(experiments_to_process[experiments_to_process['mouse_id']== mouse]['date']).sort_values(ignore_index=True)
    date_as_dt = pd.to_datetime(date)
    exp_number = sorted_exps[sorted_exps== date_as_dt].index[0]
    exp_numbers.append(exp_number)
    mice.append(mouse)
    if fiber_side == 'left':
        ipsi_cue = 'high cues'
        contra_cue = 'low cues'
    else:
        ipsi_cue = 'low cues'
        contra_cue = 'high cues'
    with open(saving_folder + save_filename, "rb") as f:
        session_kernels = pickle.load(f)
        if index == 0:
            ipsi_choice_kernel = session_kernels['kernels']['ipsi choices']
            contra_choice_kernel = session_kernels['kernels']['contra choices']
            ipsi_cue_kernel = session_kernels['kernels'][ipsi_cue]
            contra_cue_kernel = session_kernels['kernels'][contra_cue]
            reward_kernel = session_kernels['kernels']['rewards']
            no_reward_kernel = session_kernels['kernels']['no rewards']
            time_stamps = session_kernels['time_stamps']
        else:
            ipsi_choice_kernel = np.vstack([ipsi_choice_kernel, session_kernels['kernels']['ipsi choices']])
            contra_choice_kernel = np.vstack([contra_choice_kernel, session_kernels['kernels']['contra choices']])
            ipsi_cue_kernel = np.vstack([ipsi_cue_kernel, session_kernels['kernels'][ipsi_cue]])
            contra_cue_kernel = np.vstack([contra_cue_kernel, session_kernels['kernels'][contra_cue]])
            reward_kernel = np.vstack([reward_kernel, session_kernels['kernels']['rewards']])
            no_reward_kernel = np.vstack([no_reward_kernel, session_kernels['kernels']['no rewards']])


In [15]:
first_session_inds = np.where(np.asarray(exp_numbers)<3)[0].astype(int)
first_ipsi = ipsi_choice_kernel[first_session_inds]
first_contra = contra_choice_kernel[first_session_inds]
first_ipsi_cue = ipsi_cue_kernel[first_session_inds]
first_contra_cue = contra_cue_kernel[first_session_inds]
first_reward = reward_kernel[first_session_inds]
first_mice = np.asarray(mice)[first_session_inds]

In [16]:
mean_first_kernels = {}
mean_first_kernels['ipsi choices'] = np.mean(first_ipsi, axis=0)
mean_first_kernels['contra choices'] = np.mean(first_contra, axis=0)
mean_first_kernels['ipsi cues'] = np.mean(first_ipsi_cue, axis=0)
mean_first_kernels['contra cues'] = np.mean(first_contra_cue, axis=0)
mean_first_kernels['rewards'] = np.mean(first_reward, axis=0)
sem_first_kernels = {}
sem_first_kernels['ipsi choices'] = sem(first_ipsi, axis=0)
sem_first_kernels['contra choices'] = sem(first_contra, axis=0)
sem_first_kernels['ipsi cues'] = sem(first_ipsi_cue, axis=0)
sem_first_kernels['contra cues'] = sem(first_contra_cue, axis=0)
sem_first_kernels['rewards'] = sem(first_reward, axis=0)

In [26]:
fig, axs = plt.subplots(nrows=1, ncols=len(param_names), sharey=True, figsize=(11,6))
axs[0].set_ylabel('Regression coefficient', fontsize=12)
f_exp_numbers = np.asarray(exp_numbers)[first_session_inds]
num_sessions = len(f_exp_numbers)
colours = cm.viridis(np.linspace(0, 0.8, num_sessions))
for session_num in range(num_sessions):
    c=colours[exp_numbers[session_num]]
    axs[0].plot(time_stamps, first_ipsi_cue[session_num, :], color=c, label =first_mice[session_num])
    axs[1].plot(time_stamps, first_contra_cue[session_num, :], color=c)
    axs[0].set_title('cues --> ipsi ')
    axs[1].set_title('cues --> contra')
    axs[2].plot(time_stamps, first_ipsi[session_num, :], color=c)
    axs[2].set_title('ipsi choices')
    axs[3].plot(time_stamps, first_contra[session_num, :], color=c)
    axs[3].set_title('contra choices')
    axs[4].plot(time_stamps, first_reward[session_num, :], color=c)
    axs[4].set_title('rewards')
for ax in axs:
    ax.axvline(0, color='k')
axs[0].legend(loc='lower left', bbox_to_anchor= (0.5, 1.1), ncol=num_sessions,
            borderaxespad=0, frameon=False, title='Session number')

NameError: name 'param_names' is not defined

In [17]:
param_names = param_names = ['ipsi cues', 'contra cues', 'ipsi choices', 'contra choices', 'rewards', 'no rewards']
fig, axs = plt.subplots(nrows=1, ncols=len(param_names), sharey=True, figsize=(11,6))
axs[0].set_ylabel('Regression coefficient', fontsize=12)
for param_num, param_name in enumerate(param_names):
    param_kernel = mean_first_kernels[param_name]
    axs[param_num].plot(time_stamps, param_kernel, label=param_name)
    axs[param_num].axvline(0, color='k')
    axs[param_num].fill_between(time_stamps, param_kernel - sem_first_kernels[param_name] ,  param_kernel + sem_first_kernels[param_name], alpha=0.5,
                            facecolor='#7FB5B5', linewidth=0)
    axs[param_num].set_title(param_name)
    axs[param_num].set_xlabel('Time (s)', fontsize=12)
plt.tight_layout()

KeyError: 'no rewards'

In [11]:
mean_kernels = {}
mean_kernels['ipsi choices'] = np.mean(ipsi_choice_kernel, axis=0)
mean_kernels['contra choices'] = np.mean(contra_choice_kernel, axis=0)
mean_kernels['ipsi cues'] = np.mean(ipsi_cue_kernel, axis=0)
mean_kernels['contra cues'] = np.mean(contra_cue_kernel, axis=0)
mean_kernels['rewards'] = np.mean(reward_kernel, axis=0)
mean_kernels['no rewards'] = np.mean(no_reward_kernel, axis=0)

In [12]:
for a in range(0, no_reward_kernel.shape[0]):
    plt.plot(no_reward_kernel[a, :])

In [26]:
for a in range(0, reward_kernel.shape[0]):
    plt.plot(reward_kernel[a, :])

In [13]:
sem_kernels = {}
sem_kernels['ipsi choices'] = sem(ipsi_choice_kernel, axis=0)
sem_kernels['contra choices'] = sem(contra_choice_kernel, axis=0)
sem_kernels['ipsi cues'] = sem(ipsi_cue_kernel, axis=0)
sem_kernels['contra cues'] = sem(contra_cue_kernel, axis=0)
sem_kernels['rewards'] = sem(reward_kernel, axis=0)
sem_kernels['no rewards'] = sem(no_reward_kernel, axis=0)

In [14]:
param_names = param_names = ['ipsi cues', 'contra cues', 'ipsi choices', 'contra choices', 'rewards', 'no rewards']
fig, axs = plt.subplots(nrows=1, ncols=len(param_names), sharey=True, figsize=(11,6))
axs[0].set_ylabel('Regression coefficient', fontsize=12)
for param_num, param_name in enumerate(param_names):
    param_kernel = mean_kernels[param_name]
    axs[param_num].plot(time_stamps, param_kernel, label=param_name)
    axs[param_num].axvline(0, color='k')
    axs[param_num].fill_between(time_stamps, param_kernel - sem_kernels[param_name] ,  param_kernel + sem_kernels[param_name], alpha=0.5,
                            facecolor='#7FB5B5', linewidth=0)
    axs[param_num].set_title(param_name)
    axs[param_num].set_xlabel('Time (s)', fontsize=12)
plt.tight_layout()

In [20]:
fig, axs = plt.subplots(nrows=1, ncols=len(param_names), sharey=True, figsize=(11,6))
axs[0].set_ylabel('Regression coefficient', fontsize=12)
num_sessions = len(exp_numbers)
max_num_sessions = max(exp_numbers)
colours = cm.viridis(np.linspace(0, 0.8, max_num_sessions +1))
for session_num in range(num_sessions):
    c=colours[exp_numbers[session_num]]
    axs[0].plot(time_stamps, ipsi_cue_kernel[session_num, :], color=c, label = exp_numbers[session_num])
    axs[1].plot(time_stamps, contra_cue_kernel[session_num, :], color=c)
    axs[0].set_title('cues --> ipsi ')
    axs[1].set_title('cues --> contra')
    axs[2].plot(time_stamps, ipsi_choice_kernel[session_num, :], color=c)
    axs[2].set_title('ipsi choices')
    axs[3].plot(time_stamps, contra_choice_kernel[session_num, :], color=c)
    axs[3].set_title('contra choices')
    axs[4].plot(time_stamps, reward_kernel[session_num, :], color=c)
    axs[4].set_title('rewards')
    axs[5].plot(time_stamps, no_reward_kernel[session_num, :], color=c)
    axs[4].set_title('no rewards')
for ax in axs:
    ax.axvline(0, color='k')

axs[0].legend(loc='lower left', bbox_to_anchor= (0.5, 1.1), ncol=num_sessions,
            borderaxespad=0, frameon=False, title='Session number')

In [136]:
plot_filename = 'SNL_photo28_29_21_32_33_34_35_mean_regression_kernels.pdf' 
plot_save_folder = 'W:\\photometry_2AC\\plots\\Linear regression\\'
plt.savefig(plot_save_folder + plot_filename, transparent=True, optimize=True)

In [127]:
plot_filename = 'SNL_photo16_17_18_21_22_26_mean_regression_kernels.pdf' 
plot_save_folder = 'W:\\photometry_2AC\\plots\\Linear regression\\'
plt.savefig(plot_save_folder + plot_filename, transparent=True, optimize=True)

In [100]:
experiments_record_tail = experiment_record[experiment_record['recording_site'] == 'tail']

,mouse_id,date,recording_site,fiber_side
28,SNL_photo18,20200223,tail,left
30,SNL_photo18,20200226,tail,left
32,SNL_photo18,20200228,tail,left
33,SNL_photo18,20200229,tail,left
35,SNL_photo18,20200303,tail,left
37,SNL_photo18,20200305,tail,left
39,SNL_photo18,20200307,tail,left
40,SNL_photo18,20200310,tail,left
42,SNL_photo18,20200312,tail,left
43,SNL_photo18,20200313,tail,left


In [99]:
for mouse_id in experiment_record_tail['mouse_id'].unique():
    experiments = experiment_record_tail[experiment_record_tail['mouse_id'] == mouse_id]
    for index, experiment in experiments.iterrows():
        mouse = experiment['mouse_id']
        date = experiment['date']
        saving_folder = 'W:\\photometry_2AC\\processed_data\\' + mouse + '\\'
        save_filename = mouse + '_' + date + '_' + 'linear_regresion_kernels.p'
        with open(saving_folder + save_filename, "rb") as f:
            session_kernels = pickle.load(f)
            if index == 1:
                ipsi_choice_kernel = session_kernels['kernels']['ipsi choices']
                contra_choice_kernel = session_kernels['kernels']['contra choices']
                cue_kernel = session_kernels['kernels']['cues']
                reward_kernel = session_kernels['kernels']['rewards']
                time_stamps = session_kernels['time_stamps']
            else:
                ipsi_choice_kernel = np.vstack([ipsi_choice_kernel, session_kernels['kernels']['ipsi choices']])
                contra_choice_kernel = np.vstack([contra_choice_kernel, session_kernels['kernels']['contra choices']])
                cue_kernel = np.vstack([cue_kernel, session_kernels['kernels']['cues']])
                reward_kernel = np.vstack([reward_kernel, session_kernels['kernels']['rewards']])

    all_kernels={}
    all_kernels['ipsi choices'] = ipsi_choice_kernel
    all_kernels['contra choices'] = contra_choice_kernel
    all_kernels['cues'] = cue_kernel
    all_kernels['rewards'] = reward_kernel
    
    mean_kernels = {}
    mean_kernels['ipsi choices'] = np.mean(ipsi_choice_kernel, axis=0)
    mean_kernels['contra choices'] = np.mean(contra_choice_kernel, axis=0)
    mean_kernels['cues'] = np.mean(cue_kernel, axis=0)
    mean_kernels['rewards'] = np.mean(reward_kernel, axis=0)
    
    std_kernels = {}
    std_kernels['ipsi choices'] = np.std(ipsi_choice_kernel, axis=0)
    std_kernels['contra choices'] = np.std(contra_choice_kernel, axis=0)
    std_kernels['cues'] = np.std(cue_kernel, axis=0)
    std_kernels['rewards'] = np.std(reward_kernel, axis=0)

    fig, axs = plt.subplots(nrows=1, ncols=len(param_names), sharey=True, figsize=(15,8))
    axs[0].set_ylabel('Regression coefficient')
    for param_num, param_name in enumerate(param_names):
        param_kernel = all_kernels[param_name]
        axs[param_num].set_prop_cycle(plt.cycler('color',colours))
        axs[param_num].plot(time_stamps,param_kernel.T)

        axs[param_num].set_title(param_name)
        axs[param_num].set_xlabel('Time (s)')
        axs[param_num].axvline(0.2)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
30
32
33
35
37
39
40
42
43
44


array([0.87801853, 0.61416135, 1.10047466, 0.68212124, 1.1398374 ,
       0.82871293, 1.29255329, 0.77321524, 0.87099428, 0.92496414,
       0.65088095, 1.45326512, 1.19129577, 1.05429092, 0.95228183,
       0.40851538, 0.80581603, 0.6517777 , 0.15186452, 0.7016423 ,
       0.54395115, 0.51660555, 0.18614848, 0.37883117, 0.04421242,
       0.21309765, 0.49045478, 1.78318808, 0.98105266, 0.748314  ,
       0.5835751 , 0.86904679, 0.8399813 , 0.63625214, 0.7217782 ,
       0.66097678, 0.60119746, 0.58549881])

In [95]:
plt.plot(np.squeeze(contra_choice_kernel[:,first_bin]))

In [21]:
experiments.shape

NameError: name 'experiments' is not defined

In [97]:
contra_choice_kernel.shape

(38, 301)